In [117]:
import pandas as pd
import numpy as np
import requests

In [118]:
'''I used the code from this link to help obtain the scorecard data: 
https://github.com/kiseki1107/College-Scorecard-Data-Analysis/blob/master/DataCollection_Cleansing.ipynb'''

'I used the code from this link to help obtain the scorecard data: \nhttps://github.com/kiseki1107/College-Scorecard-Data-Analysis/blob/master/DataCollection_Cleansing.ipynb'

In [119]:
'''API Key received from College Scorecard website'''
api_key = '57ykVTiQxEuSYJgbWUIgVHdpxfpyATyCGfJufXnM'

In [120]:
'''Base url that we will use to  query the College Scorecard API'''
base_url = 'https://api.data.gov/ed/collegescorecard/v1/schools?'
base_url = f'{base_url}api_key={api_key}&fields='

In [121]:
'''Obtain list of query parameters'''
parameters = [
    'school.degrees_awarded.predominant=3',
    'school.operating=1',
]
# Query parameters are: predominantly bachelor's degrees awarded by university, operating schools, and 4-year schools

In [122]:
'''Create query conditions for the URL'''
parameters_url = '&'
for param in parameters:
    parameters_url += param + '&'
parameters_url = parameters_url[:-1] # remove trailing ampersand
parameters_url

'&school.degrees_awarded.predominant=3&school.operating=1'

In [123]:
'''Get fields for desired output variables for specified year'''
year = 'latest'
fields = {
    #School categories
    'School Name': 'school.name',
    'School State': 'school.state',
    'Zip Code': 'school.zip',
    'School Locale': 'school.locale' ,
    'Religious Affiliation': 'school.religious_affiliation',
    'Rate of Full Time Faculty': 'school.ft_faculty_rate',
    'Faculty Avg. Salary': 'school.faculty_salary',
    
    #Student category
    'Enrollment Size': year + '.student.size',
    'Percent Males': year + '.student.demographics.men',
    'Percent Females': year + '.student.demographics.women',
    '4 Year Retention Rate': year + '.student.retention_rate.four_year.full_time',
    'Median Househould Income': year + '.student.demographics.median_hh_income',
    'Avg. Age of Entry': year + '.student.demographics.age_entry',
    
    #Admissions category
    'Admission Rate': year + '.admissions.admission_rate.overall',
    
    #Cost Category
    'Avg. Cost' : year + '.cost.attendance.academic_year',
    'Out of State Cost': year + '.cost.tuition.out_of_state',
    'In State Cost':  year + '.cost.tuition.in_state',
    'Average Net Price (Public)': year + '.cost.avg_net_price.public',
    'Average Net Price (Private)': year + '.cost.avg_net_price.private',
    
    #Aid Category
    'Percent that Received Pell Grant': year + '.aid.pell_grant_rate',
    'Percent of Undergraduates that Receive Federal Loan': year + '.aid.federal_loan_rate',
    '90th Percentile Cumulative Loan Debt': year + '.aid.cumulative_debt.90th_percentile',
    
    #Completion Category
    '150% 4 Year Completion Rate (Pooled)': year + '.completion.completion_rate_4yr_150nt_pooled',
    
    #Repayment Category
    'Percent Repaying Not in Default 5 Years': year + '.repayment.repayment_cohort.5_year_declining_balance',
    'Three Year Default Rate': year + '.repayment.3_yr_default_rate',
    'Undergraduate Federal Loan Repayment Rate' : year + '.repayment.1_yr_db_fed_repayment.ug.rate',
    
    #Earnings Category
    'Median Earnings for Students Not Enrolled 10 Years': year + '.earnings.10_yrs_after_entry.median',
}

In [124]:
'''Construct fields parameter for query url'''
fields_url = ''
for key,val in fields.items():
    fields_url += val + ','
fields_url = fields_url[:-1] # Remove trailing comma
fields_url

'school.name,school.state,school.zip,school.locale,school.religious_affiliation,school.ft_faculty_rate,school.faculty_salary,latest.student.size,latest.student.demographics.men,latest.student.demographics.women,latest.student.retention_rate.four_year.full_time,latest.student.demographics.median_hh_income,latest.student.demographics.age_entry,latest.admissions.admission_rate.overall,latest.cost.attendance.academic_year,latest.cost.tuition.out_of_state,latest.cost.tuition.in_state,latest.cost.avg_net_price.public,latest.cost.avg_net_price.private,latest.aid.pell_grant_rate,latest.aid.federal_loan_rate,latest.aid.cumulative_debt.90th_percentile,latest.completion.completion_rate_4yr_150nt_pooled,latest.repayment.repayment_cohort.5_year_declining_balance,latest.repayment.3_yr_default_rate,latest.repayment.1_yr_db_fed_repayment.ug.rate,latest.earnings.10_yrs_after_entry.median'

In [125]:
'''College Scorecard API returns 20 records at a time, so now figure out how many records we will recieve in total'''
query_url = f'{base_url}{fields_url}{parameters_url}&page=0'
response = requests.get(query_url).json()

In [126]:
response

{'metadata': {'page': 0, 'total': 1999, 'per_page': 20},
 'results': [{'latest.student.size': 5271,
   'latest.student.demographics.men': 0.3967,
   'latest.student.demographics.women': 0.6033,
   'latest.student.retention_rate.four_year.full_time': 0.5689,
   'latest.student.demographics.median_hh_income': 49720,
   'latest.student.demographics.age_entry': 20,
   'latest.admissions.admission_rate.overall': 0.9175,
   'latest.cost.attendance.academic_year': 23053,
   'latest.cost.tuition.out_of_state': 18634,
   'latest.cost.tuition.in_state': 10024,
   'latest.cost.avg_net_price.public': 14990,
   'latest.cost.avg_net_price.private': None,
   'latest.aid.pell_grant_rate': 0.7019,
   'latest.aid.federal_loan_rate': 0.7361,
   'latest.aid.cumulative_debt.90th_percentile': 47750,
   'latest.completion.completion_rate_4yr_150nt_pooled': 0.2844,
   'latest.repayment.repayment_cohort.5_year_declining_balance': 0.3311258278,
   'latest.repayment.3_yr_default_rate': 0.176,
   'latest.repaymen

In [127]:
total_pages = response['metadata']['total']
total_pages

1999

In [128]:
'''Next, determine the nubmer of iterations it will take to get all of the responses'''
num_iter = (total_pages // 20) + 1
num_iter

100

In [129]:
'''Now, construct a pandas dataframe of the data'''
_df = []
for page_num in range(0, num_iter):
    query_url = f'{base_url}{fields_url}{parameters_url}&page={page_num}'
    response = requests.get(query_url).json()
    for i in range(len(response['results'])):
        row = {}
        for key, val in fields.items():
            if (response['results'][i][val] == None) or (response['results'][i][val]=='NaN'):#Observation is missing
                row[key] = None 
            else: # Observation is not missing
                row[key] = response['results'][i][val] 
        _df.append(row)
school_df = pd.DataFrame(_df)

In [130]:
# Recode Religious Affiliation column to be indicate Catholic College or not Catholic
school_df['Catholic'] = school_df['Religious Affiliation'].apply(lambda x: 1 if x == 30.0 else 0)
school_df.drop('Religious Affiliation', axis=1, inplace=True)

In [131]:
# Create new column for average net price for public and private institutions
_length = len(school_df['Average Net Price (Public)'])
net_price = []
for i in range(_length):
    if not pd.isna(school_df['Average Net Price (Public)'].values[i]):  # Public institution has obs. for average net price
              net_price.append(school_df['Average Net Price (Public)'].values[i])
    else: # School is either private, or has no obs. for average net price
              net_price.append(school_df['Average Net Price (Private)'].values[i])
school_df['Average Net Price'] = net_price

In [132]:
school_df[school_df['School Name'] == 'Duquesne University']

,School Name,School State,Zip Code,School Locale,Rate of Full Time Faculty,Faculty Avg. Salary,Enrollment Size,Percent Males,Percent Females,4 Year Retention Rate,...,Percent that Received Pell Grant,Percent of Undergraduates that Receive Federal Loan,90th Percentile Cumulative Loan Debt,150% 4 Year Completion Rate (Pooled),Percent Repaying Not in Default 5 Years,Three Year Default Rate,Undergraduate Federal Loan Repayment Rate,Median Earnings for Students Not Enrolled 10 Years,Catholic,Average Net Price
1247,Duquesne University,PA,15282,11,0.5171,9367.0,5837.0,0.3637,0.6363,0.8547,...,0.1923,0.6202,33250.0,0.8038,0.849254,0.025,0.909865,66207.0,1,29309.0


In [133]:
school_df

,School Name,School State,Zip Code,School Locale,Rate of Full Time Faculty,Faculty Avg. Salary,Enrollment Size,Percent Males,Percent Females,4 Year Retention Rate,...,Percent that Received Pell Grant,Percent of Undergraduates that Receive Federal Loan,90th Percentile Cumulative Loan Debt,150% 4 Year Completion Rate (Pooled),Percent Repaying Not in Default 5 Years,Three Year Default Rate,Undergraduate Federal Loan Repayment Rate,Median Earnings for Students Not Enrolled 10 Years,Catholic,Average Net Price
0,Alabama A & M University,AL,35762,12,0.7110,7709.0,5271.0,0.3967,0.6033,0.5689,...,0.7019,0.7361,47750.0,0.2844,0.331126,0.176,1.095066,36339.0,0,14990.0
1,University of Alabama at Birmingham,AL,35294-0110,12,0.7754,11049.0,13328.0,0.3890,0.6110,0.8310,...,0.3512,0.4798,37540.0,0.6090,0.571058,0.063,1.014047,46990.0,0,16953.0
2,University of Alabama in Huntsville,AL,35899,12,0.6434,9688.0,7785.0,0.5810,0.4190,0.8291,...,0.2536,0.3976,36250.0,0.5483,0.588311,0.065,0.977098,54361.0,0,15860.0
3,Alabama State University,AL,36104-0271,12,0.6501,7221.0,3750.0,0.3691,0.6309,0.6127,...,0.7627,0.8232,45000.0,0.3142,0.289705,0.180,1.094995,32084.0,0,13650.0
4,The University of Alabama,AL,35487-0100,12,0.7604,10291.0,31900.0,0.4451,0.5549,0.8693,...,0.1772,0.3802,34000.0,0.7149,0.635813,0.054,0.980264,52751.0,0,22597.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,Arizona College-Dallas,TX,75243-1200,11,0.2308,7453.0,197.0,0.0914,0.9086,NaN,...,0.6842,1.0000,NaN,NaN,NaN,0.213,1.014655,NaN,0,NaN
1995,Christ Mission College,TX,78254-1000,21,NaN,NaN,45.0,0.4222,0.5778,0.6000,...,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1996,Urshan College,MO,63385,21,NaN,5352.0,302.0,0.4338,0.5662,0.6429,...,0.0000,0.0000,NaN,0.2727,NaN,NaN,NaN,NaN,0,NaN
1997,Yeshiva Yesoda Hatorah Vetz Chaim,NY,11211-7106,11,1.0000,3543.0,140.0,1.0000,0.0000,1.0000,...,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [134]:
school_df.iloc[1990:]

,School Name,School State,Zip Code,School Locale,Rate of Full Time Faculty,Faculty Avg. Salary,Enrollment Size,Percent Males,Percent Females,4 Year Retention Rate,...,Percent that Received Pell Grant,Percent of Undergraduates that Receive Federal Loan,90th Percentile Cumulative Loan Debt,150% 4 Year Completion Rate (Pooled),Percent Repaying Not in Default 5 Years,Three Year Default Rate,Undergraduate Federal Loan Repayment Rate,Median Earnings for Students Not Enrolled 10 Years,Catholic,Average Net Price
1990,San Diego Global Knowledge University,CA,92101-7446,11,NaN,NaN,77.0,0.5584,0.4416,0.9143,...,0.0000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1991,Congregation Talmidei Mesivta Tiferes Shmiel A...,NY,11219-5440,11,0.7692,4511.0,179.0,1.0000,0.0000,0.9444,...,0.5852,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,13998.0
1992,University of Arkansas System eVersity,AR,72207-3608,12,NaN,NaN,799.0,0.3542,0.6458,NaN,...,0.0000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1993,College Unbound,RI,02905-2340,12,NaN,NaN,114.0,0.2193,0.7807,NaN,...,0.1250,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1994,Arizona College-Dallas,TX,75243-1200,11,0.2308,7453.0,197.0,0.0914,0.9086,NaN,...,0.6842,1.0,NaN,NaN,NaN,0.213,1.014655,NaN,0,NaN
1995,Christ Mission College,TX,78254-1000,21,NaN,NaN,45.0,0.4222,0.5778,0.6000,...,0.0000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1996,Urshan College,MO,63385,21,NaN,5352.0,302.0,0.4338,0.5662,0.6429,...,0.0000,0.0,NaN,0.2727,NaN,NaN,NaN,NaN,0,NaN
1997,Yeshiva Yesoda Hatorah Vetz Chaim,NY,11211-7106,11,1.0000,3543.0,140.0,1.0000,0.0000,1.0000,...,0.0000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1998,Ponce Health Sciences University-East,PR,00731-0000,12,0.5000,5052.0,41.0,0.2195,0.7805,NaN,...,NaN,NaN,NaN,NaN,NaN,0.014,NaN,NaN,0,NaN


In [135]:
school_df.describe()

,School Locale,Rate of Full Time Faculty,Faculty Avg. Salary,Enrollment Size,Percent Males,Percent Females,4 Year Retention Rate,Median Househould Income,Avg. Age of Entry,Admission Rate,...,Percent that Received Pell Grant,Percent of Undergraduates that Receive Federal Loan,90th Percentile Cumulative Loan Debt,150% 4 Year Completion Rate (Pooled),Percent Repaying Not in Default 5 Years,Three Year Default Rate,Undergraduate Federal Loan Repayment Rate,Median Earnings for Students Not Enrolled 10 Years,Catholic,Average Net Price
count,1999.000000,1846.000000,1939.000000,1997.000000,1997.000000,1997.000000,1841.000000,1751.000000,1927.000000,1640.000000,...,1971.000000,1971.000000,1770.000000,1862.000000,1781.000000,1876.000000,1868.000000,1815.000000,1999.000000,1815.000000
mean,19.532266,0.643335,7703.661166,4506.690035,0.438295,0.552191,0.743475,62707.593946,22.862999,0.682116,...,0.383832,0.508351,34336.140678,0.535013,0.652215,0.070868,0.969460,50502.958678,0.099050,19965.311295
std,9.352931,0.275347,2695.443754,7941.239449,0.186785,0.189665,0.152908,12730.058804,4.169054,0.214457,...,0.181666,0.224467,6663.840136,0.199135,0.165394,0.052185,0.074030,14666.181509,0.298803,8269.142912
min,11.000000,0.000000,527.000000,0.000000,0.000000,0.000000,0.000000,17277.000000,17.000000,0.019700,...,0.000000,0.000000,5500.000000,0.000000,0.132867,0.000000,0.319212,16792.000000,0.000000,169.000000
25%,11.000000,0.441225,6040.500000,640.000000,0.357700,0.495300,0.666700,55764.000000,20.000000,0.566150,...,0.258700,0.381150,31000.000000,0.400450,0.562543,0.034000,0.932986,41591.000000,0.000000,14286.500000
50%,13.000000,0.668450,7426.000000,1685.000000,0.429700,0.568000,0.758300,63170.000000,22.000000,0.714500,...,0.363500,0.532500,34000.000000,0.537450,0.670061,0.059000,0.980732,48106.000000,0.000000,19202.000000
75%,23.000000,0.893075,9130.000000,4553.000000,0.500800,0.638400,0.839700,71677.000000,24.000000,0.839500,...,0.484500,0.672600,37999.500000,0.670525,0.775444,0.093000,1.016714,56839.500000,0.000000,24577.500000
max,43.000000,1.000000,20988.000000,98630.000000,1.000000,1.000000,1.000000,96737.000000,51.000000,1.000000,...,1.000000,1.000000,55365.000000,1.000000,0.945070,0.370000,1.134698,132969.000000,1.000000,54902.000000


In [136]:
school_df['Catholic'].value_counts()

0    1801
1     198
Name: Catholic, dtype: int64

In [137]:
# Obtain counts of missing entries for each column
null_counts = []
for col in school_df:
    variable = {}
    count = school_df[col].isnull().sum()
    variable[col] = count
    null_counts.append(variable)
null_counts    

[{'School Name': 0},
 {'School State': 0},
 {'Zip Code': 0},
 {'School Locale': 0},
 {'Rate of Full Time Faculty': 153},
 {'Faculty Avg. Salary': 60},
 {'Enrollment Size': 2},
 {'Percent Males': 2},
 {'Percent Females': 2},
 {'4 Year Retention Rate': 158},
 {'Median Househould Income': 248},
 {'Avg. Age of Entry': 72},
 {'Admission Rate': 359},
 {'Avg. Cost': 185},
 {'Out of State Cost': 88},
 {'In State Cost': 88},
 {'Average Net Price (Public)': 1451},
 {'Average Net Price (Private)': 732},
 {'Percent that Received Pell Grant': 28},
 {'Percent of Undergraduates that Receive Federal Loan': 28},
 {'90th Percentile Cumulative Loan Debt': 229},
 {'150% 4 Year Completion Rate (Pooled)': 137},
 {'Percent Repaying Not in Default 5 Years': 218},
 {'Three Year Default Rate': 123},
 {'Undergraduate Federal Loan Repayment Rate': 131},
 {'Median Earnings for Students Not Enrolled 10 Years': 184},
 {'Catholic': 0},
 {'Average Net Price': 184}]

In [138]:
# Drop any row that has missing value for 90th percentile cumulative debt, median earnings for non-enrolled students 10 years
# after entry, 150% 4 year completion rate,
_length = len(school_df)
nan_indices = []
for i in range(_length):
    #print(school_df['90th Percentile Cumulative Loan Debt'].values[i])
    if pd.isna(school_df['90th Percentile Cumulative Loan Debt'].values[i]): nan_indices.append(i)
    elif pd.isna(school_df['Median Earnings for Students Not Enrolled 10 Years'].values[i]): nan_indices.append(i)
    elif pd.isna(school_df['150% 4 Year Completion Rate (Pooled)'].values[i]) : nan_indices.append(i)
school_df.drop(index=nan_indices, inplace=True)

In [139]:
# Obtain counts of missing entries for each column. Now, 90th Percentile Cumulative Debt should have no missing values!
null_counts = []
for col in school_df:
    variable = {}
    count = school_df[col].isnull().sum()
    variable[col] = count
    null_counts.append(variable)
null_counts  

[{'School Name': 0},
 {'School State': 0},
 {'Zip Code': 0},
 {'School Locale': 0},
 {'Rate of Full Time Faculty': 41},
 {'Faculty Avg. Salary': 32},
 {'Enrollment Size': 1},
 {'Percent Males': 1},
 {'Percent Females': 1},
 {'4 Year Retention Rate': 39},
 {'Median Househould Income': 83},
 {'Avg. Age of Entry': 0},
 {'Admission Rate': 195},
 {'Avg. Cost': 39},
 {'Out of State Cost': 5},
 {'In State Cost': 5},
 {'Average Net Price (Public)': 1130},
 {'Average Net Price (Private)': 573},
 {'Percent that Received Pell Grant': 20},
 {'Percent of Undergraduates that Receive Federal Loan': 20},
 {'90th Percentile Cumulative Loan Debt': 0},
 {'150% 4 Year Completion Rate (Pooled)': 0},
 {'Percent Repaying Not in Default 5 Years': 12},
 {'Three Year Default Rate': 0},
 {'Undergraduate Federal Loan Repayment Rate': 1},
 {'Median Earnings for Students Not Enrolled 10 Years': 0},
 {'Catholic': 0},
 {'Average Net Price': 39}]

In [140]:
school_df

,School Name,School State,Zip Code,School Locale,Rate of Full Time Faculty,Faculty Avg. Salary,Enrollment Size,Percent Males,Percent Females,4 Year Retention Rate,...,Percent that Received Pell Grant,Percent of Undergraduates that Receive Federal Loan,90th Percentile Cumulative Loan Debt,150% 4 Year Completion Rate (Pooled),Percent Repaying Not in Default 5 Years,Three Year Default Rate,Undergraduate Federal Loan Repayment Rate,Median Earnings for Students Not Enrolled 10 Years,Catholic,Average Net Price
0,Alabama A & M University,AL,35762,12,0.7110,7709.0,5271.0,0.3967,0.6033,0.5689,...,0.7019,0.7361,47750.0,0.2844,0.331126,0.176,1.095066,36339.0,0,14990.0
1,University of Alabama at Birmingham,AL,35294-0110,12,0.7754,11049.0,13328.0,0.3890,0.6110,0.8310,...,0.3512,0.4798,37540.0,0.6090,0.571058,0.063,1.014047,46990.0,0,16953.0
2,University of Alabama in Huntsville,AL,35899,12,0.6434,9688.0,7785.0,0.5810,0.4190,0.8291,...,0.2536,0.3976,36250.0,0.5483,0.588311,0.065,0.977098,54361.0,0,15860.0
3,Alabama State University,AL,36104-0271,12,0.6501,7221.0,3750.0,0.3691,0.6309,0.6127,...,0.7627,0.8232,45000.0,0.3142,0.289705,0.180,1.094995,32084.0,0,13650.0
4,The University of Alabama,AL,35487-0100,12,0.7604,10291.0,31900.0,0.4451,0.5549,0.8693,...,0.1772,0.3802,34000.0,0.7149,0.635813,0.054,0.980264,52751.0,0,22597.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1946,Milligan University,TN,37682,22,0.9907,6173.0,796.0,0.4648,0.5352,0.7429,...,0.2618,0.5356,34534.0,0.6622,0.722611,0.036,0.965127,44068.0,0,19806.0
1954,Husson University,ME,04401-2929,13,0.9778,7297.0,2642.0,0.4383,0.5617,0.7280,...,0.4189,0.7416,35750.0,0.5645,0.641234,0.080,1.008693,43582.0,0,18286.0
1963,Indiana Wesleyan University-National & Global,IN,46953,41,1.0000,7320.0,6859.0,0.2674,0.7326,NaN,...,0.4844,0.6038,35500.0,0.3754,0.615623,0.069,1.005050,54411.0,0,22207.0
1968,Purdue University Global,IN,47906,11,0.2084,4775.0,28106.0,0.3539,0.6461,1.0000,...,0.5471,0.4779,43600.0,0.2654,0.304624,0.112,1.049673,31462.0,0,NaN


In [141]:
# Construct ratio of out-of-state and in-state tuition costs
school_df['Out-In State Tuition Ratio'] = school_df['Out of State Cost'] / school_df['In State Cost']

In [142]:
# Construct ratio of meidan debt to median 10 year earnings
school_df['Debt to Earnings Ratio'] = school_df['90th Percentile Cumulative Loan Debt'] / school_df['Median Earnings for Students Not Enrolled 10 Years']

In [144]:
final_school_df = school_df.copy()

In [145]:
dropped_columns = [
    'Out of State Cost',
    'In State Cost',
    '90th Percentile Cumulative Loan Debt',
    'Median Earnings for Students Not Enrolled 10 Years',
    'Average Net Price (Public)',
    'Average Net Price (Private)'
]
final_school_df.drop(dropped_columns, axis=1)

,School Name,School State,Zip Code,School Locale,Rate of Full Time Faculty,Faculty Avg. Salary,Enrollment Size,Percent Males,Percent Females,4 Year Retention Rate,...,Percent that Received Pell Grant,Percent of Undergraduates that Receive Federal Loan,150% 4 Year Completion Rate (Pooled),Percent Repaying Not in Default 5 Years,Three Year Default Rate,Undergraduate Federal Loan Repayment Rate,Catholic,Average Net Price,Out-In State Tuition Ratio,Debt to Earnings Ratio
0,Alabama A & M University,AL,35762,12,0.7110,7709.0,5271.0,0.3967,0.6033,0.5689,...,0.7019,0.7361,0.2844,0.331126,0.176,1.095066,0,14990.0,1.858939,1.314015
1,University of Alabama at Birmingham,AL,35294-0110,12,0.7754,11049.0,13328.0,0.3890,0.6110,0.8310,...,0.3512,0.4798,0.6090,0.571058,0.063,1.014047,0,16953.0,2.380952,0.798893
2,University of Alabama in Huntsville,AL,35899,12,0.6434,9688.0,7785.0,0.5810,0.4190,0.8291,...,0.2536,0.3976,0.5483,0.588311,0.065,0.977098,0,15860.0,2.114548,0.666838
3,Alabama State University,AL,36104-0271,12,0.6501,7221.0,3750.0,0.3691,0.6309,0.6127,...,0.7627,0.8232,0.3142,0.289705,0.180,1.094995,0,13650.0,1.752439,1.402568
4,The University of Alabama,AL,35487-0100,12,0.7604,10291.0,31900.0,0.4451,0.5549,0.8693,...,0.1772,0.3802,0.7149,0.635813,0.054,0.980264,0,22597.0,2.806122,0.644538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1946,Milligan University,TN,37682,22,0.9907,6173.0,796.0,0.4648,0.5352,0.7429,...,0.2618,0.5356,0.6622,0.722611,0.036,0.965127,0,19806.0,1.000000,0.783653
1954,Husson University,ME,04401-2929,13,0.9778,7297.0,2642.0,0.4383,0.5617,0.7280,...,0.4189,0.7416,0.5645,0.641234,0.080,1.008693,0,18286.0,1.000000,0.820293
1963,Indiana Wesleyan University-National & Global,IN,46953,41,1.0000,7320.0,6859.0,0.2674,0.7326,NaN,...,0.4844,0.6038,0.3754,0.615623,0.069,1.005050,0,22207.0,1.000000,0.652442
1968,Purdue University Global,IN,47906,11,0.2084,4775.0,28106.0,0.3539,0.6461,1.0000,...,0.5471,0.4779,0.2654,0.304624,0.112,1.049673,0,NaN,1.786487,1.385799


In [146]:
school_df.to_csv('scorecard_dataset.csv', index = False)